### setup

In [ ]:
import requests
import json

token = "abcd1234" # use your real token
headers = {"Authorization": f"Token {token}"}

url_stub = "http://0.0.0.0:8000/api/v1" # or https://validation-server-stg.urban.org/api/v1/

### /api/v1/command/

#### post a new command

* `POST` upon new file upload

In [ ]:
sanitized_command_input = {
        "epsilon": 1,
        "analysis_query": "SELECT mars, COUNT(recid) as n FROM puf.puf_kueyama GROUP BY mars",
        "transformation_query": "CREATE TABLE puf.puf_kueyama AS SELECT * FROM puf.puf"
    }

payload = {
    "command_type": 2,
    "command_name" : "command.sql", 
    "sanitized_command_input": json.dumps(sanitized_command_input)
}

r = requests.post(f"{url_stub}/command/", headers=headers, data=payload)

In [ ]:
print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get a command

In [ ]:
command_id = r.json()["command_id"]
r = requests.get(f"{url_stub}/command/{command_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### rename a command

* The mockup allows you to rename a command

In [ ]:
payload = {
    "command_title" : "new_command_name", 
}
command_id = r.json()["command_id"]
r = requests.patch(f"{url_stub}/command/{command_id}/", headers=headers, data=payload)
print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get list of commands

In [ ]:
r2 = requests.get(f"{url_stub}/command/", headers=headers)

print(r2.status_code)
print(r2.reason)
for key, value in r2.json().items():
    print(key, value)

### /api/v1/synthetic-data-run/

#### post a run

* Can this be created directly in Django when a command is `POST`-ed?
* TO-DO: Trigger the backend AWS Lambda function to run the SmartNoise query

In [ ]:
payload = {
    "command_id": command_id, # from the POST to command
    "epsilon": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/synthetic-data-run/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get a synthetic data run

In [ ]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/synthetic-data-run/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/synthetic-data-result/

#### post a synthetic result

* Will be done from the backend

In [ ]:
result = {
    "status": "success",
    "data": "some_data",
    "accuracy": "some_accuracy"
}

payload = {
    "command_id": command_id,
    "run_id": run_id,
    "result": json.dumps(result),
    "privacy_budget_used": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/synthetic-data-result/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get a synthetic data result

* What is the best way to signal if a run failed?

In [ ]:
r = requests.get(f"{url_stub}/synthetic-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/confidential-data-run

#### post a confidential data run

* Will actually be multiple `POST` requests

In [ ]:
payload = {
    "command_id": command_id, # from the POST to command
    "epsilon": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/confidential-data-run/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get a confidential data run

In [ ]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-run/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/confidential-data-result/

#### post a confidential data result

* Will be done from the backend

In [ ]:
result = {
    "status": "success",
    "data": "some_data",
    "accuracy": "some_accuracy"
}

payload = {
    "command_id": command_id,
    "run_id": run_id,
    "result": json.dumps(result),
    "privacy_budget_used": 1.00,
    "display_results_decision": True,
    "release_results_decision": False
}

r = requests.post(f"{url_stub}/confidential-data-result/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get a confidential data result

In [ ]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/public-user-budget/

#### get user budget

In [ ]:
researcher_id = 1
r = requests.get(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### update user budget

In [ ]:
payload = {'total_budget_allocated': '100.00', 'privacy_budget_used': '1'}

r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### don't go over budget

In [ ]:
payload = {'total_budget_allocated': '100.00', 'privacy_budget_used': '1000'}

r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
print(r.text)